In [1]:
from tvm import relay
from tvm.relay import testing

In [2]:
def get_net(batch_size,
            num_classes=10,
            image_shape=(1, 28, 28),
            dtype="float32"):
    data_shape = (batch_size,) + image_shape
    data = relay.var("data", shape=data_shape, dtype=dtype)
    fc1 = relay.nn.dense(data, relay.var("fc1_weight"), units=128)
    fc1 = relay.nn.bias_add(fc1, relay.var("fc1_bias"), axis=-1)
    act1 = relay.nn.relu(fc1)
    fc2 = relay.nn.dense(act1, relay.var("fc2_weight"), units=64)
    fc2 = relay.nn.bias_add(fc2, relay.var("fc2_bias"), axis=-1)
    act2 = relay.nn.relu(fc2)
    fc3 = relay.nn.dense(act2, relay.var("fc3_weight"), units=num_classes)
    fc3 = relay.nn.bias_add(fc3, relay.var("fc3_bias"), axis=-1)
    mlp = relay.nn.softmax(data=fc3)
    args = relay.analysis.free_vars(mlp)
    return relay.Function(args, mlp)

In [3]:
def get_workload(batch_size,
                 num_classes=10,
                 image_shape=(1, 28, 28),
                 dtype="float32"):
    net = get_net(batch_size, num_classes, image_shape, dtype)
    return testing.create_workload(net)

In [4]:
batch_size = 1
net, params = get_workload(batch_size)

In [5]:
print(net)

v0.0.4
def @main(%data: Tensor[(1, 1, 28, 28), float32], %fc1_weight: Tensor[(128, 28), float32], %fc1_bias: Tensor[(128), float32], %fc2_weight: Tensor[(64, 128), float32], %fc2_bias: Tensor[(64), float32], %fc3_weight: Tensor[(10, 64), float32], %fc3_bias: Tensor[(10), float32]) -> Tensor[(1, 1, 28, 10), float32] {
  %0 = nn.dense(%data, %fc1_weight, units=128) /* ty=Tensor[(1, 1, 28, 128), float32] */;
  %1 = nn.bias_add(%0, %fc1_bias, axis=-1) /* ty=Tensor[(1, 1, 28, 128), float32] */;
  %2 = nn.relu(%1) /* ty=Tensor[(1, 1, 28, 128), float32] */;
  %3 = nn.dense(%2, %fc2_weight, units=64) /* ty=Tensor[(1, 1, 28, 64), float32] */;
  %4 = nn.bias_add(%3, %fc2_bias, axis=-1) /* ty=Tensor[(1, 1, 28, 64), float32] */;
  %5 = nn.relu(%4) /* ty=Tensor[(1, 1, 28, 64), float32] */;
  %6 = nn.dense(%5, %fc3_weight, units=10) /* ty=Tensor[(1, 1, 28, 10), float32] */;
  %7 = nn.bias_add(%6, %fc3_bias, axis=-1) /* ty=Tensor[(1, 1, 28, 10), float32] */;
  nn.softmax(%7) /* ty=Tensor[(1, 1, 28, 1

In [6]:
print(params)

{'fc1_weight': <tvm.NDArray shape=(128, 28), cpu(0)>
array([[ 0.01914623,  0.08440422,  0.04030711, ...,  0.05488154,
        -0.13988835,  0.1744135 ],
       [ 0.00856962, -0.03347234, -0.09234889, ..., -0.15609087,
        -0.11418793, -0.13284534],
       [ 0.06005403, -0.09676699, -0.013214  , ..., -0.03362882,
        -0.17095542,  0.07549378],
       ...,
       [ 0.05213438, -0.00916106,  0.18489547, ...,  0.02428367,
        -0.06446337, -0.01614044],
       [-0.03029653, -0.15502024, -0.13413443, ..., -0.10187755,
        -0.0685126 ,  0.10833906],
       [ 0.16205099,  0.04425791, -0.15488537, ..., -0.13206285,
        -0.17732708, -0.12114036]], dtype=float32), 'fc1_bias': <tvm.NDArray shape=(128,), cpu(0)>
array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,